<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Prepare-vvgish-input-manually" data-toc-modified-id="Prepare-vvgish-input-manually-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Prepare vvgish input manually</a></span></li><li><span><a href="#We-can-directly-go-from-path-of-mp3-file-to-input-vggish" data-toc-modified-id="We-can-directly-go-from-path-of-mp3-file-to-input-vggish-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>We can directly go from path of mp3 file to input vggish</a></span></li></ul></li><li><span><a href="#Segmenting-the-data" data-toc-modified-id="Segmenting-the-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Segmenting the data</a></span><ul class="toc-item"><li><span><a href="#Finding-matches-in-a-dataset" data-toc-modified-id="Finding-matches-in-a-dataset-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Finding matches in a dataset</a></span></li><li><span><a href="#Passing-data-thorugh-the-vvgish" data-toc-modified-id="Passing-data-thorugh-the-vvgish-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Passing data thorugh the vvgish</a></span></li></ul></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
from vggish import mel_features
from vggish import vggish_input
from vggish import vggish_slim
import executors

import numpy as np
import inspect

import jina
from jina import Document, DocumentArray



### Prepare vvgish input manually

In [21]:
import librosa

In [22]:
x_audio, sample_rate = librosa.load('data/sample.mp3')

/Users/davidbuchaca1/opt/anaconda3/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [23]:
log_mel_examples = vggish_input.waveform_to_examples(x_audio, sample_rate)

In [24]:
log_mel_examples.shape

(28, 96, 64)

### We can directly go from path of mp3 file to input vggish

In [25]:
log_mel_examples = vggish_input.mp3_to_examples('data/sample.mp3')

/Users/davidbuchaca1/opt/anaconda3/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [26]:
log_mel_examples.shape

(28, 96, 64)

In [27]:
log_mel_examples = vggish_input.wavfile_to_examples('data/Beethoven_1.wav')

In [28]:
log_mel_examples.shape, log_mel_examples.flatten().shape

((19, 96, 64), (116736,))

## Segmenting the data

The segmenter in audio-example from jina examples does the following at segment time:

- uses `read_wav(path_to_wav)` reads a wav file and returns a numpy array (`data`) and a integer (`sample_rate`)

- 

To read the data the code uses `read_wav` to read the data from a path.
This function calls `soundfile.read` which returns a numpy array `data`.



Example for `data/Beethoven_1.wav`:

- data after `soundfile.read` has shape (806912,)

- `mel_data=wav2vel(data)` has shape (806912,)




In [29]:
import soundfile as sf

wav_data, sample_rate = sf.read('data/Beethoven_1.wav', dtype='int16')
print(wav_data.shape)
wav_data = np.mean(wav_data, axis=1)
data = wav_data / sample_rate
print(data.shape)

(806912, 2)
(806912,)


In [30]:
from vggish.vggish_input import waveform_to_examples

aux = waveform_to_examples(data, sample_rate)
print(aux.shape)

aux = waveform_to_examples(data, sample_rate).squeeze()
print(aux.shape)

(19, 96, 64)
(19, 96, 64)


In [31]:
segmenter = executors.VggishSegmenter()

In [32]:
print(inspect.getsource(segmenter.segment))

    def segment(self, docs, *args, **kwargs):

        for doc in docs:
            data, sample_rate = self.read_wav(doc.uri)
            mel_data = self.wav2mel(data, sample_rate)
            for idx, blob in enumerate(mel_data):
                #self.logger.debug(f'blob: {blob.shape}')
                doc.chunks.append(Document(offset=idx, weight=1.0, blob=blob))



Note that each chunk is a Document containing each element from mel_data.

In particular this will create as elements as mel_data.shape[0]

In [33]:
aux[0].shape

(96, 64)

### Finding matches in a dataset 

We have seen that given an audio input we create a numpy array `(n, 96, 64)` where `n` depends on the input audio.

At index time we will index each segment to a vector `(n, 96, 64)` will be converted to



```
(0, 96, 64)   ->  (96, 64)  -> embedding
(1, 96, 64)   ->  (96, 64)  -> embedding
...
(n-1, 96, 64) ->  (96, 64)  -> embedding
```


Then given a query with shape `(n_q, 96,64)` we will create an embedding for each chunk in the query


```
(0, 96, 64)     ->  (96, 64)  -> embedding
(1, 96, 64)     ->  (96, 64)  -> embedding
...
(n_q-1, 96, 64) ->  (96, 64)  -> embedding
```

Afterwards we will find for each chunk embedding which is its closest match.

Given a chunk from the query $c_k$ we will compute

```
d(c_k, c) for each c 
```

Then we will get the 'hitted' parents

```
parent_id_1 : [0.1, 0.3]
parent_id_2 : [0.3, 0.4]
...
parent_id_N : [0.1]
```




In [37]:
print(inspect.getsource(segmenter.read))

    @requests(on='/search')
    def read(self, docs, *args, **kwargs):
        for doc in docs:
            data, sample_rate = self.read_wav(doc.uri)
            mel_data = self.wav2mel(data, sample_rate)
            doc.blob = mel_data[0]



In [ ]:
data_1 = vggish_input.wavfile_to_examples('data/Beethoven_1.wav')
data_2 = vggish_input.wavfile_to_examples('data/Beethoven_2.wav')
print(data_1.shape)
print(data_2.shape)

In [120]:
segmenter = executors.VggishSegmenter()

In [129]:
data, sample_rate = segmenter.read_wav('data/Beethoven_1.wav')
data.shape

(806912,)

In [141]:
data, sample_rate = segmenter.read_wav('data/Beethoven_2.wav')
data.shape

(1409024,)

### Passing data thorugh the vvgish

In [28]:
from executors import VggishEncoder

Note that different wav files will be represented with different numpy arrays:

In [86]:
data_1 = vggish_input.wavfile_to_examples('data/Beethoven_1.wav')
data_2 = vggish_input.wavfile_to_examples('data/Beethoven_2.wav')
print(data_1.shape)
print(data_2.shape)

(19, 96, 64)
(33, 96, 64)


Now let's define a VggishEncoder and look at how to encode the data

In [10]:
encoder = VggishEncoder()

/usr/local/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.9/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:329: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '


INFO:tensorflow:Restoring parameters from /Users/davidbuchaca/Documents/git_stuff/neuralsearch/jina_2/audio-search/models/vggish_model.ckpt


In [42]:
inspect.getsourcefile(encoder._encode)

'/Users/davidbuchaca/Documents/git_stuff/neuralsearch/jina_2/audio-search/executors.py'

We can take a look at how data is encoded

In [40]:
print(inspect.getsource(encoder._encode))

    def _encode(self, docs: DocumentArray, *args, **kwargs):
        blobs = docs.get_attributes('blob')
        [embedding_batch] = self.sess.run([self.embedding_tensor],
                                           feed_dict={self.feature_tensor: blobs})
        result = self.post_processor.postprocess(embedding_batch)
        embedding_matrix = (np.float32(result) - 128.) / 128.
        
        for d,e in zip(docs, embedding_matrix):
            d.embedding = e



In [51]:
encoder._encode

<bound method VggishEncoder._encode of <executors.VggishEncoder object at 0x172aed850>>

In [103]:
d1 = Document()
d2 = Document()

d1.blob = data_1
d2.blob = data_2

darray = DocumentArray([d1,d2])

In [104]:
d1.blob.shape, d2.blob.shape

((19, 96, 64), (33, 96, 64))

In [105]:
aux = darray.get_attributes('blob')

In [106]:
encoder._encode(darray)

ValueError: setting an array element with a sequence.

In [85]:
#Document(data_1)

In [12]:
encoder.encode()

AttributeError: 'VggishEncoder' object has no attribute 'encode'